Welcome to sql and python lesson 2. Last lesson, we created a basic database, added some data, tested it around, and deleted it. We used an in-memory database because we did not intend to keep the data.

In this lesson, we will expand our sql database to save data to the
disk, and to manipulate it using the Python REPL or terminal. We will stick to one table for this lesson. Next lesson, we will start learning how to use multiple tables.

Let's start by importing the sqlite3 module again.

In [1]:
#Let's load the sqlite3 module
import sqlite3 #imports the module sqlite3

In [2]:
#this time, we will create a file in our working directory to store the database.
db = sqlite3.connect('cities.db') #be sure the file ends with .db or .sql

Note: If you're using Jupyter notebooks, you should see the 'tool_cooperative.db' file in your left panel. If you are using an IDE, you can find the file using the `os` and `glob` modules.

Now let's create a cursor to connect to the database.

In [3]:
cur = db.cursor()

Let's use this occasion to execute the SQL statements that will create the database ourselves. Let us first delete the "cities" table if it exists so we can create a new one.

In [4]:
#primary key is the key we will use to retrieve items for the database.

sql_create_statement = ['create table cities (id integer primary key, name text not null, province text, \
                         country text not null, population integer)']

#'not null' means we have to have a value for that field. 

#Note: you can use capital letters if you want. Much SQL code you'll encounter is in capitals
#but using them is not required

Now that we have our sql statement, let's execute it with the `cur.execute()` statement to run it and create the cities table in the database

In [5]:
cur.execute(sql_create_statement[0])

Now let's add Villcabamba as our first city. As you all know its in Loja province, Ecuador, with a 
population of approximately 4000.

In [6]:
cur.execute('insert into cities (ID, name, province, country, population) values (1, "Villcabamba", "Loja", "Ecuador", 4000)')

We're about to retrieve the data on Villcabamba in a second, but before that, let's add Loja.

In [7]:
cur.execute('insert into cities (ID, name, province, country, population) values (2, "Loja", "Loja", "Ecuador", 4000)')

Now we have two cities in our database. Let's commit the changes to make them permanent with `db.commit()`

In [12]:
db.commit() #When we run db.commit, it saves the changes to the database.
#You don't have to run it now, but its good practice to run it when you want to save changes

In [24]:
#Now let's get the data for Villcambamba. We will start out with a simple query that retrieves all the 
#fields for it. Note that the ID field is a primary key and is created automatically.

sql = 'select * from cities where name = "Villcabamba"'
villcabamba = cur.execute(sql)
print(villcabamba.fetchall())

[(1, 'Villcabamba', 'Loja', 'Ecuador', 4000)]


In [27]:
#Let's suppose we only wanted to select Loja
sql = 'select * from cities where name = "Loja"'
loja = cur.execute(sql)
print(loja.fetchall())

[(2, 'Loja', 'Loja', 'Ecuador', 4000)]


In [29]:
#Now let's print only the names and country of Villcabamba
sql = 'select name, country from cities where name = "Villcabamba"'
villcabamba2 = cur.execute(sql)
print(villcabamba2.fetchall())

[('Villcabamba', 'Ecuador')]


In [36]:
#Now let's create functions to search for items in the database. Note that we leave the connection open
#after we add an item to the database.
def add_city(name: str, province: str, country: str, population: int) -> None: #doesn't return anything
    sql = '''insert into cities (name, province, country, population) values (?,?,?,?)'''
    cur.execute(sql, (name, province, country, population))
    

In [38]:
add_city('Quito', 'Pichincha', 'Ecuador', 2700000)

In [41]:
#Now let's look up cities with their cityID.
def find_city(id: int):
    sql = '''select id, name, province, country, population from cities
    where id = ?'''
    return cur.execute(sql, (id,)).fetchall()[0]

In [42]:
find_city(1)

(1, 'Villcabamba', 'Loja', 'Ecuador', 4000)

In [43]:
find_city(2)

(2, 'Loja', 'Loja', 'Ecuador', 4000)

In [44]:
find_city(3)

(3, 'Quito', 'Pichincha', 'Ecuador', 2700000)

In [45]:
find_city(4)

(4, 'Quito', 'Pichincha', 'Ecuador', 2700000)

Let's have a function to delete a city from our database. Be very careful when deleting things in SQL, because there is no going back after you have committed the transaction.

In [50]:
def delete_city(id: int) -> None: #type hints are optional
    sql = '''delete from cities where id = ?'''
    cur.execute(sql, (id,))

We found a mistake. It turns out we listed the inhabitants of the regions around Quito in its population instead of the city's population. Let's delete Quito and add it back with the new values.

In [52]:
delete_city(3) 

In [53]:
add_city('Quito', 'Pichincha', 'Ecuador', 1978376)

In [54]:
#use find city to make sure its added
find_city(3)

(3, 'Quito', 'Pichincha', 'Ecuador', 1978376)

That was somewhat complicated. We had to delete Quito and create it again. In the next lesson. we will learn how to update values in SQL. In the meantime, be sure and close the database properly with the `cur.close()`, `db.commit()`, and `db.close()`.

In [55]:
cur.close()
db.commit()
db.close()